In [117]:
import os
import json
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import filtering_data

In [118]:
ingre_str = '-멸치,감자,고구마,안심,마늘,계란,콩,마요네즈,오징어,머스터드'
df_food, df_material, df_process = filtering_data.filtering_data(ingre_str)

#df_food = pd.read_csv('../레시피_기본정보.csv', error_bad_lines=False, encoding="cp949")
#df_material = pd.read_csv('../레시피_재료정보.csv', error_bad_lines=False, encoding="cp949")
#df_process = pd.read_csv('../레시피_과정정보.csv', error_bad_lines=False, encoding="cp949")

In [119]:
#df_food
df_material

,레시피 코드,재료명,재료용량,재료타입명
0,1,고추장,1/2큰술,부재료
1,1,양지머리,100g,부재료
2,1,소금,약간,양념
3,1,국간장,약간,양념
4,1,다진파,약간,양념
...,...,...,...,...
5934,195453,소금,2t,주재료
5935,195453,부침가루,1/2T,주재료
5936,195453,전분,1/2T,주재료
5937,195453,계란,5개,주재료


In [126]:
dict_food = {} #전체 재료에 대한 정보

dict_main = {}  #주재료
dict_spyce = {} #부재료
dict_side = {}  #양념

dict_gram = {} #용량

for index, row in df_material.iterrows() :
    #-----------------------------------------------------
    #각 Dictionary 공간에 해당되는 재료 삽입
    if row['레시피 코드'] in dict_food:
        dict_gram[row['레시피 코드']] +=  row['재료용량'] + ','
        dict_food[row['레시피 코드']] += row['재료명'] + ','
    else :
        dict_gram[row['레시피 코드']] =  row['재료용량'] + ','
        dict_food[row['레시피 코드']] =  row['재료명'] + ','
        
        #빈 Dicotionary 공간 생성
        dict_main[row['레시피 코드']] =  ''
        dict_side[row['레시피 코드']] =  ''
        dict_spyce[row['레시피 코드']] = ''

    if row['재료타입명'] == '주재료' :
        dict_main[row['레시피 코드']] += row['재료명'] + ','
        
    elif row['재료타입명'] == '부재료' :
        dict_side[row['레시피 코드']] += row['재료명'] + ','
        
    elif row['재료타입명'] == '양념' :
        dict_spyce[row['레시피 코드']] +=  row['재료명'] + ','
    #------------------------------------------------------

    
#-----------------------------------------------------------
#pandas 기본정보에 정보를 입력하기 위해 각 공간 할당
df_food['재료'] = ''
df_food['재료용량'] = ''
df_food['주재료'] = ''
df_food['부재료'] = ''
df_food['양념'] = ''

for key, value in dict_food.items():
    food_value = value.split(',')
    food_value = food_value[:-1] #마지막 열 제거 ','
    dict_food[key] = food_value
    
    gram_value = dict_gram[key].split(',')
    gram_value = gram_value[:-1]
    dict_gram[key] =  gram_value
        
    #주재료-----------------------------------------
    try :
        main_value = dict_main[key].split(',')
    except :
        main_value = ['']
    #부재료-----------------------------------------
    try :
        side_value = dict_side[key].split(',')
    except :
        side_value = ['']
    #양념-------------------------------------------
    try :
        spyce_value = dict_spyce[key].split(',')
    except :
        spyce_value = ['']
    #--------------------------------------------- 
    #마지막 리스트에 존재하는 ''요소 삭제
    dict_main[key] =  main_value[:-1]
    dict_side[key] =  side_value[:-1]
    dict_spyce[key] = spyce_value[:-1]

food_list = []
gram_list = []
main_list = []
side_list = []
spyce_list = []

#----------------------------------------------------
#신규 COLUMN에 포함되는 값들을 LIST에 저장
#----------------------------------------------------
for index, row in df_food.iterrows() :
    food_list.append(dict_food[row['레시피 코드']] )
    gram_list.append(dict_gram[row['레시피 코드']] )
    main_list.append(dict_main[row['레시피 코드']] )
    side_list.append(dict_side[row['레시피 코드']] )
    spyce_list.append(dict_spyce[row['레시피 코드']] )

#신규 COLUMN 작성
df_food['재료'] = food_list
df_food['주재료'] = main_list
df_food['부재료'] = side_list
df_food['양념'] = spyce_list
df_food['재료용량'] = gram_list


In [127]:
all_material_list = df_material['재료명'].unique() #모든 요리들의 재료 리스트
vector_index_list = []  #요리가 가지고 있는 재료들을 비교한 리스트 값
vector_set_list = []    #각 백터들의 설정값

for index ,row in df_food.iterrows():
    #해당 요리에 포함되는 모든 재료
    food_material = row['재료']
    food_index = []
    
    #주재료, 부재료, 양념에 대한 각 가중치 Value
    each_material = [row['주재료'],row['부재료'],row['양념']]
    each_weight_list = [6,4,2]#가중치 임의값
    
    #벡터 공간
    vector = [0]* len(all_material_list) 
    
    #요리 재료 순회, 각 재료의 재료 타입에 따른 가중치 입력
    for data in food_material : 
        data = data.strip()
        try :
            index = np.where(all_material_list == data)[0][0]
            food_index.append(index)
            for i in range(3) :
                if data in each_material[i] :
                    vector[index] = each_weight_list[i]
        except :
            print('잘못된 재료 입력입니다 : {}'.format(data))
    vector_index_list.append(food_index)
    vector_set_list.append(vector)
    
    
df_food['재료 인덱스'] = vector_index_list
df_food['VECTOR'] = vector_set_list


In [128]:
#메뉴와 메뉴에 대한 비교후 점수 리턴
def compare_menu(menu1_vector, menu2_vector) :
    score = 0
    for index in range(len(menu1_vector)) :
        if menu1_vector[index] != 0 and menu2_vector[index] != 0 :
            score += menu1_vector[index]
        elif menu1_vector[index] == 0 and menu2_vector[index] != 0 :
            score += 0
            #score -= menu2_vector[index] // 2 #부족한 재료에 대해 -처리(임의 상수값)
    return score

#레시피 메뉴 코드에 대해 메뉴 추천
def process_menu_code(df, code, menu_cnt) :
    try :
        cur_menu_vector = df[df['레시피 코드'] == code]['VECTOR'].item()
    except :
        print('해당 음식은 현재 조건에 만족되지 않은 메뉴입니다')
        return
    comp_dict = {}
    for index, row in df.iterrows():
        if row['레시피 코드'] == code :
            continue
        comp_menu_vector = row['VECTOR']
        score = compare_menu(cur_menu_vector, comp_menu_vector)
        
        #점수 리스트 
        comp_dict[row['레시피 코드']] = score
    
    cur_menu_name = df[df['레시피 코드']== code]['레시피 이름']#.item()
    print('현재 메뉴 : {}'.format(cur_menu_name))
    
    #Dictionary의 큰값순서대로 정렬된 리스트를 만들어, Menu cnt만큼 추천 요리 표시
    comp_list = sorted(comp_dict.items(), key=lambda x: x[1], reverse= True)
    for index in range(menu_cnt):
        #comp_list[index][0] = Key, comp_list[index][1] = score
        try : #메뉴 수 초과 가능성 염두
            recommand_menu = df[df['레시피 코드']== comp_list[index][0]]['레시피 이름'].item()
            print('추천 메뉴 {} :'.format(index+1) ,recommand_menu)    
        except :
            return
    return

#유저 입력에 대해 메뉴 추천
def process_user_input(df,user_list,menu_cnt) :
    #유저가 입력한 재료 리스트 표시
    print('유저 입력:', user_list)
    
    #유저 입력에 대한 백터 공간 설정
    user_vector = [0]* len(all_material_list)
    for data in user_list :
        try :
            index = np.where(all_material_list == data)[0][0]
            user_vector[index] = 1
        except :
            print('잘못된 재료 입력입니다 : {}'.format(data))
            
    #유저가 가지고 있는 재료에 대해 각 요리들을 비교
    comp_dict = {}
    for index, row in df.iterrows():
        comp_food = row['VECTOR']
        
        #유저의 입력에는 주재료, 부재료에 대한 가중치가 없어서, 반대로 비교를 수행한다.
        score = compare_menu(comp_food, user_vector) 
        comp_dict[row['레시피 코드']] = score
    
    #Dictionary의 큰값순서대로 정렬된 리스트를 만들어, Menu cnt만큼 추천 요리 표시
    comp_list = sorted(comp_dict.items(), key=lambda x: x[1], reverse= True)
    for index in range(menu_cnt):
        #comp_list[index][0] = Key, comp_list[index][1] = score
        try : #메뉴 수 초과 가능성 염두
            recommand_menu = df[df['레시피 코드']== comp_list[index][0]]['레시피 이름'].item()
            print('추천 메뉴 {} :'.format(index+1) ,recommand_menu)
        except :
            return
    return 


In [130]:
temp_input = ingre_str.split(',')
for index, data in enumerate(temp_input) :
    if '-' in data :
        del temp_input[index]

process_user_input(df_food,temp_input,5)
process_menu_code(df_food, 85, 5)


df_return = df_food.loc[:, ['레시피 코드','재료','재료용량']]
df_return

유저 입력: ['감자', '고구마', '안심', '마늘', '계란', '콩', '마요네즈', '오징어', '머스터드']
추천 메뉴 1 : 나물비빔밥
추천 메뉴 2 : 오징어탕수
추천 메뉴 3 : 식빵고구마파이
추천 메뉴 4 : 쇠고기장조림
추천 메뉴 5 : 컬리플라워해물샐러드
현재 메뉴 : 84    갈비탕
Name: 레시피 이름, dtype: object
추천 메뉴 1 : 쇠고기무국
추천 메뉴 2 : 배추속댓국
추천 메뉴 3 : 무맑은국
추천 메뉴 4 : 알탕
추천 메뉴 5 : 곰탕


,레시피 코드,재료,재료용량
0,1,"[고추장, 양지머리, 소금, 국간장, 다진파, 다진마늘, 참기름, 설탕, 쌀, 안심...","[1/2큰술, 100g, 약간, 약간, 약간, 약간, 약간, 약간, 4컵, 200g..."
1,2,"[수수, 차조, 콩, 팥, 소금, 멥쌀, 찹쌀]","[1컵, 1컵, 1/2컵, 1/2컵, 약간, 1컵, 2컵]"
2,3,"[호박, 당근, 청고추, 홍고추, 밥, 다진파, 다진마늘, 진간장, 참기름, 소금,...","[1/4개, 1/2개, 약간, 약간, 3공기, 2큰술, 1큰술, 4큰술, 1큰술, ..."
3,4,"[파, 마늘, 참기름, 깨소금, 고춧가루, 진간장, 소금, 통후추, 쌀, 콩나물, ...","[1/2개, 2쪽, 1큰술, 1큰술, 조금, 4큰술, 약간, 약간, 2컵, 300g..."
7,8,"[쇠고기, 감자, 양파, 당근, 완두콩, 우유, 소금, 통후추, 카레, 밥]","[200g, 2개, 1개, 1/2개, 4큰술, 1/2컵, 약간, 약간, 70g, 4공기]"
...,...,...,...
531,120476,"[들기름, 간장, 후추, 배추, 미나리, 계란, 다진마늘, 다진생강, 다진돼지고기,...","[1T, 1T, 약간, 10장, 10줄기, 1개, 1/2T, 2t, 100g, 10..."
532,180330,"[고구마, 식빵, 마요네즈, 꿀, 소금, 건블루베리, 우유, 계란노른자]","[2개(大), 5쪽, 1T, 1/2T, 약간, 1/2T, 1컵, 1개]"
533,180344,"[카레가루, 물, 밥, 방울토마토, 양파, 계란, 셀러리, 햄, 후추]","[3T, 280ml, 1공기, 15알, 1/8개, 2개, 1대, 50g, 약간]"
535,195428,"[두부, 식초, 설탕, 마요네즈, 깨, 밥, 오이, 당근, 가지, 어린잎채소, 김,...","[2.5T, 1/2T, 1/2T, 1T, 1/2T, 3공기, 1/2개, 1/2개, ..."
